In [29]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import os
import sys
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, grandparent_dir, "src", "utils")
sys.path.append(utils_dir)
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, "src", "utils")
sys.path.append(utils_dir)

In [30]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from process_data import process_data
from weighted_accuracy import weighted_accuracy_scorer
from plot_learning_curves import plot_learning_curves
from weighted_accuracy_and_tools import decompose_y, reconstruct_y


In [31]:
X = os.path.join("..", "..","data","original_input", "X_train_filled.csv")
Y = os.path.join("..", "..","data","original_input", "X_test_GgyECq8.csv")
W = os.path.join("..", "..","data","original_input", "y_train_jJtXgMX.csv")

X_train = pd.read_csv(X, delimiter=',')
X_test = pd.read_csv(Y,delimiter=',')
y_train = pd.read_csv(W, delimiter=',')
delivery_start = X_test["DELIVERY_START"]

In [32]:
X_train = process_data(X_train.copy(deep=True), "predicted_spot_price", None, "standard")
X_test = process_data(X_test.copy(deep=True), "predicted_spot_price", None, "standard")
y_train = process_data(y_train.copy(deep=True), None, None, None)

In [33]:
y_train_direction, y_train_magnitude = decompose_y(y_train['spot_id_delta'])

In [34]:
from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier(n_jobs=-1)

# Set up the parameter grid to search
# Here, we are varying the number of estimators (trees) and the maximum depth of trees
param_grid = {
    #'n_estimators': [100, 200, 300],  # Number of trees in the forest
    #'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train_direction)

best_params_clf = grid_search.best_params_

# Best parameters found
print("Best parameters found: ", grid_search.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best parameters found:  {'min_samples_leaf': 4, 'min_samples_split': 10}


In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Initialize the RandomForestRegressor
reg = RandomForestRegressor(n_jobs=-1)

# Set up the parameter grid to search
param_grid = {
    #'n_estimators': [100, 200, 300],  # Number of trees in the forest
    #'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]     # Minimum number of samples required to be at a leaf node
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=reg, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Assuming X_train and y_train_magnitude are already defined
# Fit GridSearchCV
grid_search.fit(X_train, y_train_magnitude)

best_params_rgr = grid_search.best_params_

# Best parameters found
print("Best parameters found: ", grid_search.best_params_)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best parameters found:  {'min_samples_leaf': 2, 'min_samples_split': 5}


In [39]:
clf =  RandomForestClassifier(n_jobs=-1, **best_params_clf)
clf.fit(X_train, y_train_direction)
y_pred_dir = clf.predict(X_test)

rgr =  RandomForestRegressor(n_jobs=-1, **best_params_rgr)
rgr.fit(X_train, y_train_magnitude)
y_pred_magn = rgr.predict(X_test)

In [40]:
y_pred = reconstruct_y(y_pred_dir, y_pred_magn)

In [41]:
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred.reshape(-1)}
y_pred = pd.DataFrame(data = d)

y_pred.to_csv("../../data/output_y_test/y_result_double_rd_forest7.csv", index = False)